In [1]:
import sys
import sqlite3
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import datetime as dt

pd.options.plotting.backend = "plotly"
sys.path.append('../src')

from constants import DB_PATH, NEWEST_DATA_QUERY, ALL_DATA_QUERY, GOALS_PATH
from utils import LinearRegression

In [25]:
def make_momentum_line(df:pd.DataFrame)->pd.DataFrame:
    frame = df.copy()
    frame = frame.loc[frame.Type.str.contains(r'slope_\d+',regex=True)]
    windows = frame.Type.str.extract(r'slope_(\d+)',expand = False).astype(int)
    
    mx_window = windows.max()
    mn_window = windows.min()

    mx_window = frame.loc[frame.Type == f'slope_{mx_window}'].sort_values('Date').reset_index(drop = True)
    mn_window = frame.loc[frame.Type == f'slope_{mn_window}'].sort_values('Date').reset_index(drop = True)

    res = pd.DataFrame(data = {'Date':mx_window.Date,'Weight':mx_window.Weight - mn_window.Weight})
    res['Type'] = 'momentum'
    return pd.concat([df,res],axis=0)


conn = sqlite3.connect(DB_PATH)
df = (pd.read_sql_query(NEWEST_DATA_QUERY, conn)
        .assign(dateTime=lambda x: pd.to_datetime(x['dateTime']))
        .rename(columns={'dateTime': 'Date'})
        .sort_values('Date'))

goals = pd.read_excel(GOALS_PATH,sheet_name='Weight Goal',usecols='D:G')
actualVgoal = df['Date weight'.split()].merge(goals, on='Date', how = 'outer',suffixes=('_actual', '_goal'))

fig = (
actualVgoal['Date weight_actual weight_goal'.split()]
    .melt(id_vars='Date', var_name='Type', value_name='Weight')
    .pipe(lambda x: pd.concat([x,make_trend_line(df,7).query("Type == 'slope_7'")],axis=0))
    .pipe(lambda x: pd.concat([x,make_trend_line(df,14)],axis=0))
    .pipe(make_momentum_line)
    .pipe(lambda x: pd.concat([x,make_tangent_line(actualVgoal,7)],axis=0))
    .pipe(lambda x: pd.concat([x,make_tangent_line(actualVgoal,14)],axis=0))
    .pipe(lambda x: pd.concat([x,make_tangent_line(actualVgoal,21)],axis=0))
    .assign(defict_indication = lambda x: x.Type.str.contains('slope'))
    .plot(x='Date', y='Weight', kind='line', color='Type', facet_row='defict_indication',width = 1600, height = 1000)
    .update_traces(selector = dict(name = 'weight_actual'), line = dict(color = 'blue'),opacity = 0.25)
    .update_traces(selector = dict(name = 'smooth_trend'), line = dict(color = 'blue'))
    .update_traces(selector = dict(name = 'weight_goal'), line = dict(dash = 'dash', color = 'black'))
    .update_traces(selector=dict(name='weight_trend_7'), opacity=1.0, line = dict(color = 'darkred'))
    .update_traces(selector=dict(name='weight_trend_14'), opacity=0.7, line = dict(color = 'firebrick'))
    .update_traces(selector=dict(name='weight_trend_21'), opacity=0.4, line = dict(color = 'salmon'))
    .update_traces(selector=dict(name='slope'), opacity=0.25, line = dict(color = 'green'))
    .add_hline(y = -1000, line_dash = 'dash', line_color = 'red', row = 1,opacity = 0.5)
    .add_hline(y = -500, line_dash = 'dash', line_color = 'red', row = 1,opacity = 0.25)
    .add_hline(y = 0, line_dash = 'dash', line_color = 'red', row = 1,opacity = 0.125)
    .add_hline(y = 500, line_dash = 'dash', line_color = 'red', row = 1,opacity = 0.25)
    .add_hline(y = 1000, line_dash = 'dash', line_color = 'red', row = 1,opacity = 0.5)
    .update_layout(
        xaxis = dict(showgrid = False),
        xaxis2 = dict(showgrid = False),
        yaxis = dict(showgrid = False, showticklabels = True, title = '', zeroline = False),
        yaxis2 = dict(showgrid = False, showticklabels = True, title = '', zeroline = False),
        showlegend = False,
        title = 'Weight and Goals over Time',
        title_x = 0.5,
    )
    .update_yaxes(matches=None)
    .update_annotations(text = ''))

In [21]:
make_trend_line(df,7)

,Date,Type,Weight
0,2024-12-01,smooth_trend,80.429298
1,2024-12-02,smooth_trend,80.583567
2,2024-12-03,smooth_trend,80.659691
3,2024-12-04,smooth_trend,80.741936
4,2024-12-05,smooth_trend,80.828755
...,...,...,...
71,2025-01-03,slope_7,-845.639814
72,2025-01-04,slope_7,-878.007953
73,2025-01-05,slope_7,-1161.826723
74,2025-01-06,slope_7,-1515.896712


In [26]:
fig

In [ ]:
# conn = sqlite3.connect(DB_PATH)
# c = conn.cursor()

# df = (pd.read_sql_query(NEWEST_DATA_QUERY, conn)
#         .assign(dateTime=lambda x: pd.to_datetime(x['dateTime']))
#         .rename(columns={'dateTime': 'Date'})
#         .sort_values('Date'))

In [2]:
def getStepsChart(df: pd.DataFrame) -> go.Figure:
   return  (df.plot(x='Date', y = 'steps', kind = 'bar')
               .add_hline(y = 10_000, line_dash = 'dash', line_color = 'red')
               .update_layout(
                              xaxis = dict(showgrid = False),
                              xaxis2 = dict(showgrid = False),
                              yaxis = dict(showgrid = False, showticklabels = True, title = '', zeroline = False),
                              yaxis2 = dict(showgrid = False, showticklabels = True, title = '', zeroline = False),
                              showlegend = False,
                              title = 'Step count over time',
                              title_x = 0.5,
                        )
                  .update_yaxes(matches=None)
                  .update_annotations(text = ''))

In [3]:
def getNutritionChart(df: pd.DataFrame) -> go.Figure:
    # Create figure
    fig = go.Figure()

    # Add stacked bars for macros
    fig.add_trace(go.Bar(
        x=df['Date'], y=df['protein'], name='Protein',
        marker_color='green', hoverinfo='y'
    ))

    fig.add_trace(go.Bar(
        x=df['Date'], y=df['carbs'], name='Carbs',
        marker_color='blue', hoverinfo='y'
    ))
    fig.add_trace(go.Bar(
        x=df['Date'], y=df['fat'], name='Fat',
        marker_color='orange', hoverinfo='y'
    ))


    # Add line for calories (secondary y-axis)
    fig.add_trace(go.Scatter(
        x=df['Date'], y=df['calories'].apply(lambda x: None if x == 0 else x), name='Calories',
        mode='lines', line=dict(color='red', width=2),
        yaxis='y2'
    ))

    # Add red horizontal line for goal
    fig.add_shape(
        type='line', x0=df['Date'].min(), x1=df['Date'].max(),
        y0=2000, y1=2000, line=dict(color='red', dash='dash'),
        yref='y2'  # Reference secondary y-axis
    )

    fig.add_shape(
        type='line', x0=df['Date'].min(), x1=df['Date'].max(),
        y0=150, y1=150, line=dict(color='green', dash='dash'),
        yref='y1'  # Reference y-axis
    )

    # Update layout for dual y-axes and appearance
    fig.update_layout(
        title='Macros and Calories Over Time',
        xaxis=dict(title='Date'),
        yaxis=dict(title='Macros (g)', side='left'),
        yaxis2=dict(
            title='Calories', overlaying='y', side='right'
        ),
        barmode='stack',
        legend=dict(title='Legend', orientation='h'),
    )
    #remove gridlines
    return fig.update_layout(
                        xaxis_showgrid=False,
                        yaxis_showgrid=False,
                        yaxis2_showgrid=False,
                        xaxis = dict(showgrid = False),
                        xaxis2 = dict(showgrid = False),
                        yaxis = dict(showgrid = False, showticklabels = True, title = '', zeroline = False),
                        yaxis2 = dict(showgrid = False, showticklabels = True, title = '', zeroline = False),
                        showlegend = False,
                        title = 'Macros and Calories over Time',
                        title_x = 0.5,
        )



In [14]:

def make_tangent_line(actualVgoal:pd.DataFrame,lookback: int)-> pd.DataFrame:
    '''
    makes a line of best fit for the actual weight data over the past `lookback` days to
    be displayed on the bodyweight graph long with the goals.
    '''
    domain = actualVgoal.copy().loc[lambda s: s.Date.between(dt.datetime.today()-dt.timedelta(lookback),
                                                    dt.datetime.today()+dt.timedelta(lookback)),
                            ['Date','weight_actual']]\
                        .assign(Time = lambda x: (x.Date - x.Date.min()).dt.days)
    training = domain.copy().dropna()

    lr = LinearRegression(training.Time.values.reshape(-1,1),training.weight_actual.values)
    lr.fit()

    domain[f'weight_trend_{lookback}'] = lr.predict(domain.Time.values.reshape(-1,1))
    return   (domain.drop(columns = ['weight_actual','Time'])
                    .melt(id_vars='Date', var_name='Type', value_name='Weight'))

def make_trend_line(df:pd.DataFrame,window: int)-> pd.DataFrame:
    '''
    makes a smoothed trendline for the weight data over all time with a window of 
    be displayed on the bodyweight graph long with the goals
    '''
    date = df.Date.min()
    min_date = df.Date.min()
    domain = df.copy()['Date weight'.split()].assign(Time = lambda x: (x.Date - x.Date.min()).dt.days)
    smooth_trend = []
    slope = []
    while date <= df.Date.max():
        smoothing_window = domain.copy().loc[lambda s: s.Date.between(date-dt.timedelta(window),date+dt.timedelta(window))]
        date += dt.timedelta(1)
        lr = LinearRegression(smoothing_window.Time.values.reshape(-1,1),smoothing_window.weight.values)
        lr.fit()
        smooth_trend.append(lr.predict(np.array([[(date-min_date).days]]))[0])
        slope.append(lr.beta[1])
    domain['smooth_trend'] = smooth_trend
    domain[f'slope_{window}'] = slope
    domain[f'slope_{window}'] = domain[f'slope_{window}']*3600*2.20462
    return (domain.drop(columns = ['Time','weight'])
                    .melt(id_vars='Date', var_name='Type', value_name='Weight'))

# GoalsPath = r"H:\Desktop1123\Learning\09 - Python Projects\02 FitBot\DashBoard\assets\Goals.xlsx"

def getBodyWeightChart(df:pd.DataFrame):
    goals = pd.read_excel(GOALS_PATH,sheet_name='Weight Goal',usecols='D:G')
    actualVgoal = df['Date weight'.split()].merge(goals, on='Date', how = 'outer',suffixes=('_actual', '_goal'))

    fig = (
    actualVgoal['Date weight_actual weight_goal'.split()]
        .melt(id_vars='Date', var_name='Type', value_name='Weight')
        .pipe(lambda x: pd.concat([x,make_trend_line(df,14)],axis=0))
        .pipe(lambda x: pd.concat([x,make_tangent_line(actualVgoal,7)],axis=0))
        .pipe(lambda x: pd.concat([x,make_tangent_line(actualVgoal,14)],axis=0))
        .pipe(lambda x: pd.concat([x,make_tangent_line(actualVgoal,21)],axis=0))
        .assign(defict_indication = lambda x: x.Type == 'slope')
        .plot(x='Date', y='Weight', kind='line', color='Type', facet_row='defict_indication',width = 1600, height = 1000)
        .update_traces(selector = dict(name = 'weight_actual'), line = dict(color = 'blue'),opacity = 0.25)
        .update_traces(selector = dict(name = 'smooth_trend'), line = dict(color = 'blue'))
        .update_traces(selector = dict(name = 'weight_goal'), line = dict(dash = 'dash', color = 'black'))
        .update_traces(selector=dict(name='weight_trend_7'), opacity=1.0, line = dict(color = 'darkred'))
        .update_traces(selector=dict(name='weight_trend_14'), opacity=0.7, line = dict(color = 'firebrick'))
        .update_traces(selector=dict(name='weight_trend_21'), opacity=0.4, line = dict(color = 'salmon'))
        .update_traces(selector=dict(name='slope'), opacity=0.25, line = dict(color = 'green'))
        .add_hline(y = -1000, line_dash = 'dash', line_color = 'red', row = 1,opacity = 0.5)
        .add_hline(y = -500, line_dash = 'dash', line_color = 'red', row = 1,opacity = 0.25)
        .add_hline(y = 0, line_dash = 'dash', line_color = 'red', row = 1,opacity = 0.125)
        .add_hline(y = 500, line_dash = 'dash', line_color = 'red', row = 1,opacity = 0.25)
        .add_hline(y = 1000, line_dash = 'dash', line_color = 'red', row = 1,opacity = 0.5)
        .update_layout(
            xaxis = dict(showgrid = False),
            xaxis2 = dict(showgrid = False),
            yaxis = dict(showgrid = False, showticklabels = True, title = '', zeroline = False),
            yaxis2 = dict(showgrid = False, showticklabels = True, title = '', zeroline = False),
            showlegend = False,
            title = 'Weight and Goals over Time',
            title_x = 0.5,
        )
        .update_yaxes(matches=None)
        .update_annotations(text = ''))
    return fig

In [ ]:
import dash
from dash import dcc, html
import plotly.graph_objs as go
import numpy as np

# Create the Dash app
app = dash.Dash(__name__)

# get data 
conn = sqlite3.connect(DB_PATH)
c = conn.cursor()
initialDataFrame = (pd.read_sql_query(NEWEST_DATA_QUERY, conn)
        .assign(dateTime=lambda x: pd.to_datetime(x['dateTime']))
        .rename(columns={'dateTime': 'Date'})
        .sort_values('Date'))
conn.close()

app.layout = html.Div([
    html.Div(id = 'title',children='FitBot Dashboard'),
    html.Div(
        dcc.Graph(id='graph1', figure=getBodyWeightChart(initialDataFrame.copy()),config={'displayModeBar': False}), 
        style={
            'border': '2px solid black',  # Black border
            'padding': '10px',           # Padding inside the box
            'margin-bottom': '20px'      # Space between the boxes
        }
    ),
    html.Div(
        dcc.Graph(id='graph2', figure=getNutritionChart(initialDataFrame.copy()),config={'displayModeBar': False}), 
        style={
            'border': '2px solid black',  # Black border
            'padding': '10px',           # Padding inside the box
            'margin-bottom': '20px'      # Space between the boxes
        }
    ),
    html.Div(
        dcc.Graph(id='graph3', figure=getStepsChart(initialDataFrame.copy()),config={'displayModeBar': False}), 
        style={
            'border': '2px solid black',  # Black border
            'padding': '10px',           # Padding inside the box
            'margin-bottom': '20px'      # Space between the boxes
        }
    ),
    dcc.Interval(
        id='interval-component',
        interval=10000,  # Update every second
        n_intervals=0
    )
])

# Define the callback to update the graphs
@app.callback(
    [dash.dependencies.Output('graph1', 'figure'),
     dash.dependencies.Output('graph2', 'figure'),
     dash.dependencies.Output('graph3', 'figure'),
     dash.dependencies.Output('title', 'children')],
    [dash.dependencies.Input('interval-component', 'n_intervals')]
)
def update_graphs(n):
    conn = sqlite3.connect(DB_PATH)
    _df = (pd.read_sql_query(NEWEST_DATA_QUERY, conn)
            .assign(dateTime=lambda x: pd.to_datetime(x['dateTime']))
            .rename(columns={'dateTime': 'Date'})
            .sort_values('Date'))
    conn.close()
    fig1, fig2, fig3 = getBodyWeightChart(_df), getNutritionChart(_df), getStepsChart(_df)
    return fig1, fig2, fig3, str(n)

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [12]:
import sqlite3
import pandas as pd
import sys
sys.path.append('../src')

from constants import DB_PATH
q = '''
SELECT fitbit.*
FROM fitbit
JOIN (
    SELECT dateTime, MAX(scrapeTime) AS maxScrapeTime
    FROM fitbit
    GROUP BY dateTime
) grouped
ON fitbit.dateTime = grouped.dateTime AND fitbit.scrapeTime = grouped.maxScrapeTime
WHERE fitbit.dateTime > '{}'
'''.format('2024-11-30')

conn = sqlite3.connect(DB_PATH)
c = conn.cursor()
df = pd.read_sql_query(q, conn)
df.dateTime.min()

'2024-12-01'

In [ ]:
import sys
import pandas as pd


sys.path.append('..')

goals = pd.read_excel(r'H:\Desktop1123\Learning\09 - Python Projects\02 FitBot\DashBoard\assets\Goals.xlsx',sheet_name='Weight Goal',usecols='D:G')

In [ ]:
import datetime as dt
goals.loc[goals.Date == dt.datetime.today().strftime('%Y-%m-%d'),'weight'].values[0]

81.88160810350215

: 